In [3]:
import numpy as np
import pandas as pd

input_file = "Hotel_Reviews.csv"

# Load Hotel_reviews data
df = pd.read_csv(input_file, header = 0)
data_pandas = pd.DataFrame(df)

df['is_train'] = np.random.uniform(0,1,len(df)) <= 0.75

score_bucket = []
i=0
for row in data_pandas.iloc[:,12]:
    if row > 8:
        score_bucket.append(5)
    elif row > 6:
        score_bucket.append(4)
    elif row > 4:
        score_bucket.append(3)
    elif row > 2:
        score_bucket.append(2)
    elif row > 0:
        score_bucket.append(1)
    i = i + 1

df['processed_score'] = score_bucket


In [11]:
mylist = list(df)
del mylist[0:20]
mylist

for word in mylist:
    print(word)

use
booking
hotel
small
room
floor
booked
window
asked
double
day
check
time
view
noise
negative
rooms
nice
bit
tea
coffee
bar
door
shower
staff
clean
bed
water
noisy
old
night
told
price
good
great
work
breakfast
restaurant
pay
bathroom
facilities
hot
bad
morning
open
parking
need
little
people
stay
reception
area
expensive
cold
quite
poor
better
service
food
location
wifi
air
far
beautiful
excellent
fantastic
lovely
amazing
modern
place
spacious
quiet
friendly
positive
big
walk
walking
distance
comfy
comfortable
large
close
city
super
easy
loved
helpful
extremely
definitely
restaurants
metro
station
central
perfect
wonderful
beds
best
free
value
size
convenient
london
tube


In [21]:
list_bucket=[]

              # Prepositions
neutral = set(["with","at", "from", "into", "during", "including", "until", "against", "among", "throughout", "despite",
              "towards", "upon", "of", "to", "in", "for", "on", "by", "about", "like", "through", "over", "before", "between",
              "after", "since", "without", "under", "within", "along", "following", "across", "behind", "beyond", "plus", "except",
              "but", "up", "out", "around", "down", "off", "above", "near",
              # Conjuctions
              "and", "or", "but", "nor", "so", "for", "yet", "after", "although", "as", "because", "before", "even", "though",
              "once", "since", "till", "unless", "until", "what", "when", "whenever", "wherever", "whether", "while",
              # Pronouns
              "i", "you", "he", "she", "it", "we", "they", "me", "him", "her", "us", "them", "what", "who", "this", "that",
              "these", "those",
               # etc
               "a", "all" ,"an", "am", "be", "can", "did", "didn", "get", "got", "had", "have", "if", "my", "next", "our", "some",
               "the", "was", "were", "would", "your", "just", "like", "really", "wasn"
              ])

from sklearn.feature_extraction.text import CountVectorizer
#Finding most important words in Negative Reviews
negative_comment = df['Negative_Review']
cv = CountVectorizer(analyzer = "word",stop_words = 'english',max_features = 70,ngram_range=(1,1))
most_negative_words = cv.fit_transform(negative_comment)
temp1_counts = most_negative_words.sum(axis=0)
temp1_words = cv.vocabulary_

cv2 = CountVectorizer(analyzer = "word",stop_words = 'english',max_features = 70,ngram_range=(1,1))
positive_comment = df['Positive_Review']
most_positive_words = cv2.fit_transform(positive_comment)
temp2_counts = most_positive_words.sum(axis=0)
temp2_words = cv2.vocabulary_

for row in data_pandas.iloc[:,6]:
    mylist = row.split()
    no_nums = [x for x in mylist if not (x.isdigit() or x[0] == '-' and x[1:].isdigit())]
    
    # Filter out caps
    no_nums = map(str.lower, no_nums)
    
    row_list={}
    temp_words = {**temp1_words, **temp2_words}

    for word in temp_words:
        if word in neutral:
            continue
        elif word.lower() not in no_nums:
            row_list[word] = 0
        else:
            row_list[word] = 1
       
    list_bucket.append(row_list)        
            


In [22]:
dict = {}
for entry in list_bucket:
    for key in entry:
        dict[key] = []
    break    
for entry in list_bucket:
    for key in entry:
            dict[key].append(entry[key])
            

In [23]:
for entry in list_bucket:
    for key in entry:
        df[key] = dict[key]
    break  
#data_pandas.head(10)

In [29]:
df.to_csv('preprocessed.csv')

In [27]:
from sklearn.naive_bayes import GaussianNB
from numpy import genfromtxt

# df['is_train'] = np.random.uniform(0,1,len(df)) <= 0.75
# define training and test sets
train = df[df['is_train']==True]
test = df[df['is_train']==False]

trainTargets = np.array(train['processed_score']).astype(int)
testTargets = np.array(test['processed_score']).astype(int)

# columns you want to model
features = df.columns[20:159]

# call Gaussian Naive Bayesian class with default parameters
gnb = GaussianNB()

# train model
#y_gnb = gnb.fit(train[features], trainTargets).predict(train[features])
y_gnb = gnb.fit(train[features], trainTargets).predict(test[features])


In [28]:
from sklearn.metrics import accuracy_score

accuracy_score(testTargets, y_gnb)
#gnb.predict([[0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0]])

0.65128835501338145